In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs import sim

In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        # feature map size is 7*7 by pooling
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64*7*7)   # reshape Variable
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
    def forward_test(self, x):
        res = {}
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        res["layer_1"] = x
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        res["layer_2"] = x
        x = x.view(-1, 64*7*7)   # reshape Variable
        res["layer_3"] = x        
        x = F.relu(self.fc1(x))
        res["layer_4"] = x        
        x = F.dropout(x, training=self.training)
        res["layer_5"] = x        
        x = self.fc2(x)
        res["layer_6"] = x        
        return res
    
model = MnistModel()
model

In [ ]:
batch_size = 50
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=1000)

In [ ]:
# Define the regularization strength
weight_decay = 0.001
#optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=weight_decay)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
model.train()
train_loss = []
train_accu = []
i = 0

#loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
for epoch in range(5):
    for data, target in train_loader:
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss= loss_fn(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.item())
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/batch_size*100
        train_accu.append(accuracy)
        if i % 1000 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.item(), accuracy))
        i += 1

In [ ]:
model.eval()
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    loss = F.nll_loss(output, target)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('\nTest set: \tLoss: {:.3f}\tAccuracy: {:.3f}'.format(loss, 100. * correct / len(test_loader.dataset)))

<h1>Hyperdimensional Encoding</h1>

In [ ]:
import numpy as nd

'''
r_proj = nd.random.randint(2, size=(10000,10000))
r_proj[r_proj == 0] = -1
r_inv_proj = nd.linalg.pinv(r_proj)

print(r_proj.shape, r_inv_proj.shape)

with open('proj.npy', 'wb') as f:
    nd.save(f, r_proj)
    
with open('inv.npy', 'wb') as f:
    nd.save(f, r_inv_proj)  
'''

with open('proj.npy', 'rb') as f:
    r_proj = nd.load(f)

with open('inv.npy', 'rb') as f:
    r_inv_proj = nd.load(f)    

def get_enc_model(model):
    arr, slist = sim.get_net_arr(model)

    rem = nd.zeros(10000- (len(arr) % 10000))
    if len(arr) % 10000 != 0:
        arr = nd.concatenate((arr, rem), axis=None)

    #enc_model = []
    enc_model = nd.array([])
    index = 0
    while index < len(arr):
        #enc_model.append(arr[index:index+10000] @ r_proj)
        enc_model = nd.concatenate((enc_model, (arr[index:index+10000] @ r_proj)), axis = None)
        index = index + 10000
        #print(index)

    return enc_model

def get_dec_model(enc_model):
    arr, slist = sim.get_net_arr(model)
    
    rem = nd.zeros(10000- (len(arr) % 10000))
    if len(arr) % 10000 != 0:
        arr = nd.concatenate((arr, rem), axis=None)
    
    dec_model = nd.zeros(len(arr))
    index = 0
    while index < len(arr):
        dec = enc_model[index: index + 10000] @ r_inv_proj
        dec_model[index: index + 10000] = dec
        index = index + 10000
        #print(index)

    dec_model = sim.get_arr_net(model, dec_model, slist)
    return dec_model

In [ ]:
enc_model = get_enc_model(model)
print(enc_model)

In [ ]:
print(len(enc_model))

In [ ]:
dec_model = get_dec_model(enc_model)
print(dec_model)

In [ ]:
data, target = Variable(data, volatile=True), Variable(target)
output = dec_model(data)
print(output[0].argmax(), target[0])

<h1>Homomorphic Encryption</h1>

In [ ]:
import tenseal as ts

def create_ctx():
    pmd = 8192
    cmbs = [60, 40, 40, 60]
    ctx = ts.context(ts.SCHEME_TYPE.CKKS, pmd, cmbs)
    ctx.global_scale = 2 ** 40
    ctx.generate_galois_keys()
    return ctx

context = create_ctx()

HE = Pyfhel()
HE.contextGen(scheme='ckks', n=8192, scale=2**26, qi_sizes=[31]+ [scale_power]*n_mults +[31])
HE.keyGen()
HE.relinKeyGen()

In [ ]:
a = [5,10]
b = [10,20]

enc_a = ts.ckks_vector(context, a)
enc_a

enc_b = ts.ckks_vector(context, b)
enc_b

print((enc_a + enc_b).decrypt())

In [ ]:
arr, slist = sim.get_net_arr(model)
enc_ckks_model = ts.ckks_vector(context, arr)

In [ ]:
dec_ckks_model = sim.get_arr_net(model, nd.array(enc_ckks_model.decrypt()), slist)
data, target = Variable(data, volatile=True), Variable(target)
output = dec_ckks_model(data)
print(output[0].argmax(), target[0])

In [ ]:
enc_ckks_model = (enc_ckks_model + enc_ckks_model)
dec_ckks_model = sim.get_arr_net(model, nd.array(enc_ckks_model.decrypt())/2, slist)
data, target = Variable(data, volatile=True), Variable(target)
output = dec_ckks_model(data)
print(output[0].argmax(), target[0])

In [ ]:
target_batch = next(iter(test_loader))
target_images, target_labels = target_batch

img = target_images[0]
lbl = target_labels[0]

img2 = target_images[2]
lbl2 = target_labels[2]

print(img.shape, lbl)

with torch.no_grad():
    out = model.forward_test(img.unsqueeze(0))
    
#print(out)

In [ ]:
actvs = 0
for k, v in out.items():
    print (len(v.view(-1)))
    actvs = actvs + v.numel()
print(actvs)

In [ ]:
arr, _ = sim.get_net_arr(model)
print(len(arr))

In [ ]:
model

In [ ]:
# a dict to store the activations
activation = {}
def getActivation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# register forward hooks on the layers of choice
h1 = model.conv1.register_forward_hook(getActivation('conv1'))
h2 = model.conv2.register_forward_hook(getActivation('conv2'))
h3 = model.fc1.register_forward_hook(getActivation('fc1'))
h4 = model.fc2.register_forward_hook(getActivation('fc2'))

out = model(img.unsqueeze(0))
print(activation['conv1'].numel(), activation['conv2'].numel(), activation['fc1'].numel(), activation['fc2'].numel())
#print(activation['conv2'],activation['conv2'].shape)
print(activation['conv1'].shape,activation['conv2'].shape)
# detach the hooks
h1.remove()
h2.remove()
h3.remove()
h4.remove()

In [ ]:
'''
activation = {}
hooks = {}
for name, module in model.named_modules():
    print(module)
    hooks[name] = module.register_forward_hook(getActivation(module))

output = model(img.unsqueeze(0))

print(activation, activation['conv1'].numel())

for name, _ in hooks.items():
    hooks[name].remove()
'''

In [ ]:
import numpy

arr = numpy.arange(60).reshape(3, 4, 5)
arr

In [ ]:
def find_index(index, dim = (3, 4, 5)):
    row = int(index / (dim[1] * dim[2]))
    rem = index - row * (dim[1] * dim[2])
    col = int(rem / dim[2])
    hei = rem - col * dim[2]
    return row, col, hei

find_index(40)